<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/m_cite_par_bn_v_4_class_tapt_multi_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from time import time
start_time = time()

In [6]:
!pip install datasets
!pip install transformers[torch]
!pip install scikit-learn
!pip install adapters

  Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached

In [7]:
from sklearn.metrics import accuracy_score,  f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [8]:
from datasets import load_dataset
dataset_name = "BigTMiami/citation_intent_dataset"
dataset = load_dataset(dataset_name)
dataset

Generating train split:   0%|          | 0/1688 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/139 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1688
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 139
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 114
    })
})

In [9]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
#Label Settings
id2label = {0: 'Background', 1: 'Uses', 2: 'CompareOrContrast', 3: 'Extends', 4: 'Motivation', 5: 'Future'}
label2id = {'Background': 0, 'Uses': 1, 'CompareOrContrast': 2, 'Extends': 3, 'Motivation': 4, 'Future': 5}

In [11]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="remove_output_dir",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0003,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=10, # NEED TO TRAIN FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=1, # Saves latest 2 checkpoints
    push_to_hub=False,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    load_best_model_at_end=True, # Set to false - we want the last trained model like the paper
    metric_for_best_model="f1_macro", # Is default
    # torch_compile=True,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    seed=0,
)

In [13]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper

In [14]:
import gc
from adapters import AdapterTrainer

load_name = f"BigTMiami/m_cite_par_bn_v_4_pretrain_adapter"

In [15]:
all_train_results = []
all_eval_results = []

In [16]:
for seed in range(5):
  print(f"Seed: {seed}")
  print("=========================================")
  training_args.seed = seed

  # Load Pretrained adapter without head
  loaded_adapter_name = model.load_adapter(load_name, with_head=False)

  # Add head for classification modeling
  model.add_classification_head(
      loaded_adapter_name,
      num_labels=6,
      id2label=id2label,
      overwrite_ok=True
      )

  # Set the adapter to be used for training
  model.train_adapter(loaded_adapter_name)

  adapter_hub_name = f"m_cite_par_bn_v_4_help_class_adp_S_{seed}"

  summary = model.adapter_summary()
  print(summary)

  trainer = AdapterTrainer(
      model=model,
      args=training_args,
      train_dataset=dataset["train"],
      eval_dataset=dataset["dev"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks=[early_stopping]
  )

  results = trainer.train()
  print(results)
  all_train_results.append(results)

  eval_results = trainer.evaluate(dataset["test"])
  print(eval_results)
  all_eval_results.append(eval_results)

  model.push_adapter_to_hub(
      adapter_hub_name,
      loaded_adapter_name,
      datasets_tag=dataset_name
  )
  print(f"Pushed {adapter_hub_name}")

  # Delete the seed adapter
  model.set_active_adapters(None)
  model.delete_adapter(loaded_adapter_name)

  gc.collect()

Seed: 0


(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

{'eval_loss': 0.9409617781639099, 'eval_accuracy': 0.7553956834532374, 'eval_f1_macro': 0.662391971925239, 'eval_runtime': 0.2824, 'eval_samples_per_second': 492.127, 'eval_steps_per_second': 31.864, 'epoch': 10.0}


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

Pushed m_cite_par_bn_v_4_help_class_adp_S_4


In [17]:
for result in all_train_results:
  print(result)

for result in all_eval_results:
  print(result)

TrainOutput(global_step=1060, training_loss=0.691328767560563, metrics={'train_runtime': 79.3874, 'train_samples_per_second': 212.628, 'train_steps_per_second': 13.352, 'total_flos': 1072890490917168.0, 'train_loss': 0.691328767560563, 'epoch': 10.0})
TrainOutput(global_step=1060, training_loss=0.6284047000813034, metrics={'train_runtime': 78.3699, 'train_samples_per_second': 215.389, 'train_steps_per_second': 13.526, 'total_flos': 1067973478193664.0, 'train_loss': 0.6284047000813034, 'epoch': 10.0})
TrainOutput(global_step=1060, training_loss=0.6222779444928439, metrics={'train_runtime': 78.5159, 'train_samples_per_second': 214.988, 'train_steps_per_second': 13.5, 'total_flos': 1073796367297248.0, 'train_loss': 0.6222779444928439, 'epoch': 10.0})
TrainOutput(global_step=1060, training_loss=0.7222882162849859, metrics={'train_runtime': 78.7889, 'train_samples_per_second': 214.243, 'train_steps_per_second': 13.454, 'total_flos': 1076762585769696.0, 'train_loss': 0.7222882162849859, 'epo

In [18]:
end_time = time()
total_time = end_time - start_time
print(f"Total Script Runtime: {total_time / 60:.0f}  minutes OR {total_time:.0f}  seconds")

Total Script Runtime: 9  minutes OR 535  seconds


In [19]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()


Disconnecting Session
